# Collection value evolution

Libs

In [1]:
import datetime as dt
import json
import numpy as np
import pandas as pd
import requests
import sqlite3
import yaml
from requests.exceptions import HTTPError
import time

Configuration file

In [2]:
with open(r'../config.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

url_discogs_api = "https://api.discogs.com"

## Retrieve the collection items

Get number of pages of collection items

In [17]:
try:
    query = {'page': 1, 'per_page': 100}
    url_request = url_discogs_api + "/users/" + config["discogs_user"] + "/collection/folders/0/releases"
    response = requests.get(url_request, params=query)
    response.raise_for_status()
    jsonResponse = response.json()
except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

no_pages = jsonResponse["pagination"]["pages"] # * 50

Get the collection items

In [18]:
collection_items = []
for i in range(1, no_pages + 1):
    try:
        query = {'page': i, 'per_page': 100}
        url_request = url_discogs_api + "/users/" + config["discogs_user"] + "/collection/folders/0/releases"
        response = requests.get(url_request, params=query)
        jsonResponse = response.json()
        collection_items.append(pd.json_normalize(jsonResponse["releases"]))
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

df_collection = pd.concat(collection_items, ignore_index=True)

In [5]:
df_collection.head()

,id,instance_id,date_added,rating,basic_information.id,basic_information.master_id,basic_information.master_url,basic_information.resource_url,basic_information.thumb,basic_information.cover_image,basic_information.title,basic_information.year,basic_information.formats,basic_information.labels,basic_information.artists,basic_information.genres,basic_information.styles
0,9438069,470722404,2020-05-16T20:50:48-07:00,0,9438069,417099,https://api.discogs.com/masters/417099,https://api.discogs.com/releases/9438069,,,Passenger,2016,"[{'name': 'Vinyl', 'qty': '1', 'descriptions':...","[{'name': '[PIAS]', 'catno': 'LHLP002', 'entit...","[{'name': 'Lisa Hannigan', 'anv': '', 'join': ...","[Pop, Folk, World, & Country]",[Folk]
1,9331164,877584547,2021-12-09T03:03:16-08:00,0,9331164,60848,https://api.discogs.com/masters/60848,https://api.discogs.com/releases/9331164,,,Sunset Mission,2016,"[{'name': 'Vinyl', 'qty': '2', 'descriptions':...","[{'name': '[PIAS]', 'catno': 'PIASD5013LP', 'e...","[{'name': 'Bohren & Der Club Of Gore', 'anv': ...","[Electronic, Jazz]",[Dark Jazz]
2,6239920,739755133,2021-06-24T16:19:33-07:00,0,6239920,281597,https://api.discogs.com/masters/281597,https://api.discogs.com/releases/6239920,,,Burma Shave,2008,"[{'name': 'DVD', 'qty': '1', 'text': '4:3', 'd...","[{'name': '3rd story production', 'catno': 'MC...","[{'name': 'Tom Waits', 'anv': '', 'join': '', ...","[Rock, Blues]",[]
3,1965832,470725568,2020-05-16T21:00:19-07:00,0,1965832,20096,https://api.discogs.com/masters/20096,https://api.discogs.com/releases/1965832,,,The Bad Seed,1983,"[{'name': 'Vinyl', 'qty': '1', 'descriptions':...","[{'name': '4AD', 'catno': '151.102', 'entity_t...","[{'name': 'The Birthday Party', 'anv': '', 'jo...","[Electronic, Rock]","[New Wave, Punk, Darkwave]"
4,22078003,1028746508,2022-05-13T06:54:41-07:00,0,22078003,2501125,https://api.discogs.com/masters/2501125,https://api.discogs.com/releases/22078003,,,Dragon New Warm Mountain I Believe In You,2022,"[{'name': 'Vinyl', 'qty': '2', 'text': 'Random...","[{'name': '4AD', 'catno': '4AD0408LP', 'entity...","[{'name': 'Big Thief', 'anv': '', 'join': '', ...","[Rock, Folk, World, & Country]",[Indie Rock]


In [40]:
test = df_collection['basic_information.artists'].apply(pd.Series)
df = pd.DataFrame.from_dict(test)
df.head(200)


,0,1,2,3,4
0,"{'name': 'Lisa Hannigan', 'anv': '', 'join': '...",NaN,NaN,NaN,NaN
1,"{'name': 'Bohren & Der Club Of Gore', 'anv': '...",NaN,NaN,NaN,NaN
2,"{'name': 'Tom Waits', 'anv': '', 'join': '', '...",NaN,NaN,NaN,NaN
3,"{'name': 'The Birthday Party', 'anv': '', 'joi...",NaN,NaN,NaN,NaN
4,"{'name': 'Big Thief', 'anv': '', 'join': '', '...",NaN,NaN,NaN,NaN
...,...,...,...,...,...
195,"{'name': 'Frank Sinatra', 'anv': '', 'join': '...","{'name': 'Billy May And His Orchestra', 'anv':...",NaN,NaN,NaN
196,"{'name': 'Peggy Lee', 'anv': '', 'join': '', '...",NaN,NaN,NaN,NaN
197,"{'name': 'The Beach Boys', 'anv': '', 'join': ...",NaN,NaN,NaN,NaN
198,"{'name': 'Conway Savage', 'anv': '', 'join': '...",NaN,NaN,NaN,NaN


Subselection of columns

In [6]:
selected_columns = df_collection.columns[~df_collection.columns.isin([ "basic_information.thumb", "basic_information.cover_image",\
     "basic_information.artists", "basic_information.labels", "basic_information.formats", "basic_information.genres", "basic_information.styles"])]
df_collection = df_collection[selected_columns]

Write stuff to sqlite db

In [7]:
db = sqlite3.connect("test.db")
df_collection.to_sql(name="collection", con=db, if_exists='replace')
db.close()

## Retrieve marketplace values

In [8]:
query = {'curr_abbr': 'EUR', 'token': config['discogs_token']}

collection_items_value = []
for i in df_collection.index:
    url_request = url_discogs_api + "/marketplace/stats/" + str(df_collection['id'][i])
    try:
        response = requests.get(url_request, params=query)
        response.raise_for_status()

        df_item = pd.json_normalize(response.json())
        df_item['id'] = str(df_collection['id'][i])
        df_item['time_value_retrieved'] = dt.datetime.now()
        df_item = df_item.loc[:, df_item.columns != 'lowest_price']
        collection_items_value.append(df_item)

    except HTTPError as http_err:
        if response.status_code == 429:
            time.sleep(60)
    except Exception as err:
        print(f'Other error occurred: {err}')
        
df_collection_value = pd.concat(collection_items_value, ignore_index=True)

Other error occurred: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


In [9]:
df_collection_value.head()

,num_for_sale,blocked_from_sale,lowest_price.value,lowest_price.currency,id,time_value_retrieved
0,3,False,64.99,EUR,9438069,2022-05-06 18:40:18.493661
1,26,False,24.83,EUR,9331164,2022-05-06 18:40:19.349922
2,5,False,4.99,EUR,6239920,2022-05-06 18:40:19.795479
3,1,False,34.99,EUR,1965832,2022-05-06 18:40:20.194728
4,10,False,26.98,EUR,403287,2022-05-06 18:40:20.639484


Add to database

In [10]:
db = sqlite3.connect("test.db")
df_collection_value.to_sql(name="collection_value", con=db, if_exists='append')
db.close()

## Retrieve release statistics

In [7]:
query = {'token': config['discogs_token']}

collection_items_stats = []
for i in df_collection.index:
    url_request = url_discogs_api + "/releases/" + str(df_collection['id'][i])
    try:
        response = requests.get(url_request, params=query)
        response.raise_for_status()

        df_item = pd.json_normalize(response.json())
        df_item['id'] = str(df_collection['id'][i])
        df_item['time_value_retrieved'] = dt.datetime.now()
        df_item.head()
        collection_items_stats.append(df_item)

    except HTTPError as http_err:
        if response.status_code == 429:
            time.sleep(60)
    except Exception as err:
        print(f'Other error occurred: {err}')
        
df_collection_stats = pd.concat(collection_items_stats, ignore_index=True)

In [8]:
df_collection_stats.head()

,id,status,year,resource_url,uri,artists,artists_sort,labels,series,companies,...,community.have,community.want,community.rating.count,community.rating.average,community.submitter.username,community.submitter.resource_url,community.contributors,community.data_quality,community.status,time_value_retrieved
0,9438069,Accepted,2016,https://api.discogs.com/releases/9438069,https://www.discogs.com/release/9438069-Lisa-H...,"[{'name': 'Lisa Hannigan', 'anv': '', 'join': ...",Lisa Hannigan,"[{'name': '[PIAS]', 'catno': 'LHLP002', 'entit...",[],"[{'name': 'Play It Again Sam', 'catno': '', 'e...",...,268,132,17,4.76,FuzznDistortion,https://api.discogs.com/users/FuzznDistortion,"[{'username': 'FuzznDistortion', 'resource_url...",Needs Vote,Accepted,2022-06-06 12:34:59.523234
1,9331164,Accepted,2016,https://api.discogs.com/releases/9331164,https://www.discogs.com/release/9331164-Bohren...,"[{'name': 'Bohren & Der Club Of Gore', 'anv': ...",Bohren & Der Club Of Gore,"[{'name': '[PIAS]', 'catno': 'PIASD5013LP', 'e...",[],"[{'name': '[PIAS]', 'catno': '', 'entity_type'...",...,3715,1851,496,4.72,Turboderrik,https://api.discogs.com/users/Turboderrik,"[{'username': 'Turboderrik', 'resource_url': '...",Correct,Accepted,2022-06-06 12:34:59.952067
2,6239920,Accepted,2008,https://api.discogs.com/releases/6239920,https://www.discogs.com/release/6239920-Tom-Wa...,"[{'name': 'Tom Waits', 'anv': '', 'join': '', ...",Tom Waits,"[{'name': '3rd story production', 'catno': 'MC...",[],[],...,33,20,0,0.00,Admane,https://api.discogs.com/users/Admane,"[{'username': 'Admane', 'resource_url': 'https...",Needs Vote,Accepted,2022-06-06 12:35:00.409825
3,1965832,Accepted,1983,https://api.discogs.com/releases/1965832,https://www.discogs.com/release/1965832-The-Bi...,"[{'name': 'The Birthday Party', 'anv': '', 'jo...","Birthday Party, The","[{'name': '4AD', 'catno': '151.102', 'entity_t...",[],"[{'name': 'Hansa Tonstudios', 'catno': '', 'en...",...,111,174,13,4.54,watzmann,https://api.discogs.com/users/watzmann,"[{'username': 'watzmann', 'resource_url': 'htt...",Needs Vote,Accepted,2022-06-06 12:35:00.759743
4,22078003,Accepted,2022,https://api.discogs.com/releases/22078003,https://www.discogs.com/release/22078003-Big-T...,"[{'name': 'Big Thief', 'anv': '', 'join': '', ...",Big Thief,"[{'name': '4AD', 'catno': '4AD0408LP', 'entity...",[],"[{'name': '4AD Ltd.', 'catno': '', 'entity_typ...",...,2189,264,113,4.50,phapharra,https://api.discogs.com/users/phapharra,"[{'username': 'phapharra', 'resource_url': 'ht...",Correct,Accepted,2022-06-06 12:35:01.565803


In [11]:
selected_columns = df_collection_stats.columns[df_collection_stats.columns.isin([ "id", "community.have", "community.want", "community.rating.count",\
    "community.rating.average", "time_value_retrieved"])]
df_collection_stats = df_collection_stats[selected_columns]
df_collection_stats.head()

,id,community.have,community.want,community.rating.count,community.rating.average
0,9438069,268,132,17,4.76
1,9331164,3715,1851,496,4.72
2,6239920,33,20,0,0.00
3,1965832,111,174,13,4.54
4,22078003,2189,264,113,4.50


In [13]:
type(df_collection_stats['id'])

pandas.core.series.Series

Add to database

In [ ]:
db = sqlite3.connect("test.db")
df_collection_stats.to_sql(name="collection_stats", con=db, if_exists='append')
db.close()